## Welcome to your notebook.


#### Run this cell to connect to your GIS and get started:

In [ ]:
# from arcgis.gis import GIS
# import contextlib, io
# with contextlib.redirect_stderr(io.StringIO()):
#     gis = GIS("home")

#### Now you are ready to start!

In [ ]:
# =============================================================================
# ### NWS Point Forecast to ArcGIS Feature Layer Updater ###
#
# Optional: ArcGIS Online Sync
# To publish updates to ArcGIS Online, set:
#   USE_ARCGIS=1
#   NWS_FORECAST_LAYER_ID (or FEATURE_LAYER_ITEM_ID)
# Then re-run the notebook from Cell 1.
# =============================================================================
# This script is designed to run in an ArcGIS Online Notebook environment.
# It fetches the latest 7-day weather forecast from the National Weather
# Service (NWS) API for a specific point and updates a corresponding
# Feature Layer in ArcGIS Online.
#
# --- PROCESS ---
# 1.  Connects to the ArcGIS Online organization ("home").
# 2.  Defines file paths for the configuration files located on the AGOL server.
# 3.  Reads 'PR Alert Data Sources.xlsx' to get the NWS API endpoint and
#     the latitude/longitude from the 'Parameters_or_Selectors' column.
# 4.  Performs the two-step NWS API request to get the forecast data.
# 5.  Processes the JSON response into a pandas DataFrame.
# 6.  Converts datetime fields to a string format compatible with AGOL.
# 7.  Iterates through the DataFrame to create a list of arcgis.features.Feature
#     objects, manually creating the point geometry for each.
# 8.  Connects to the target Feature Layer using its Item ID.
# 9.  Deletes all existing records in the layer.
# 10. Appends the new forecast records to the Feature Layer.
# =============================================================================

import pandas as pd
import requests
import json
import sys
import logging
import os
from pathlib import Path

# Set USE_ARCGIS=1 to enable ArcGIS Online sync; otherwise run locally.
USE_ARCGIS = os.environ.get("USE_ARCGIS", "").lower() in ("1", "true", "yes")

if USE_ARCGIS:
    from arcgis.gis import GIS
    from arcgis.features import FeatureLayer, Feature
    from arcgis.geometry import Point


In [ ]:
# --- Step 0: Configure Logging ---
# Helper: resolve local files without hardcoding machine-specific paths
def resolve_file(filename, env_var=None, search_roots=None):
    if env_var:
        env_val = os.environ.get(env_var)
        if env_val:
            return env_val
    roots = search_roots or [Path.cwd(), Path.cwd().parent, Path.home()]
    arcgis_home = Path("/arcgis/home")
    if arcgis_home.exists():
        roots.append(arcgis_home)
    for root in roots:
        if root.exists():
            match = next(root.rglob(filename), None)
            if match:
                return str(match)
    raise FileNotFoundError("Set the required env var or place the file under the repo or /arcgis/home.")

# Set up logging to provide detailed, play-by-play output
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s", stream=sys.stdout)

# --- ArcGIS Online Configuration (optional) ---
if USE_ARCGIS:
    logging.info("--- Connecting to ArcGIS Online ---")
    try:
        gis = GIS("home")
        logging.info("Connected to ArcGIS Online.")
    except Exception as e:
        logging.error(f"❌ FATAL ERROR: Could not connect to ArcGIS Online. Reason: {e}")
        sys.exit()
else:
    gis = None
    logging.info("ArcGIS disabled; running locally only.")

# --- Configuration ---
ITEM_ID = os.environ.get('NWS_FORECAST_LAYER_ID') or os.environ.get('FEATURE_LAYER_ITEM_ID')
LAYER_COUNT = 0

if USE_ARCGIS and not ITEM_ID:
    raise ValueError("Set NWS_FORECAST_LAYER_ID (or FEATURE_LAYER_ITEM_ID) in the environment.")

CONFIG_FILE_NAME = resolve_file('PR Alert Data Sources.xlsx', env_var='PR_ALERT_XLSX')
TARGET_SOURCE_NAME = 'NWS - Point Forecast Lookup'

# Local outputs (for non-ArcGIS runs)
OUTPUT_DIR = Path(os.environ.get("OUTPUT_DIR", "outputs"))
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
OUTPUT_CSV = OUTPUT_DIR / "nws_forecast.csv"
OUTPUT_GEOJSON = OUTPUT_DIR / "nws_forecast.geojson"

logging.info("--- Configuration ---")
logging.info("  > Config file resolved.")


In [ ]:
# --- Step 1: Load the configuration file ---
logging.info("\n--- Reading Configuration File ---")
try:
    config_df = pd.read_excel(CONFIG_FILE_NAME)
    nws_source_row = config_df[config_df['Source_Name'] == TARGET_SOURCE_NAME]

    if nws_source_row.empty:
        logging.error(f"❌ FATAL ERROR: Could not find a source named '{TARGET_SOURCE_NAME}' in '{CONFIG_FILE_NAME}'.")
        sys.exit()

    source_details = nws_source_row.iloc[0]
    logging.info("✅ Successfully found API configuration.")

except FileNotFoundError:
    logging.error("FATAL ERROR: Configuration file not found. Set PR_ALERT_XLSX or place PR Alert Data Sources.xlsx under the repo or /arcgis/home.")
    sys.exit()
except Exception as e:
    logging.error(f"❌ FATAL ERROR: An error occurred while reading the Excel file: {e}")
    sys.exit()

# --- Step 2: Extract details from the configuration ---
endpoint_url = source_details['URL_Endpoint']
params_or_selectors = source_details['Parameters_or_Selectors']
lat, lon = None, None

try:
    params = json.loads(params_or_selectors)
    lat = params.get('lat')
    lon = params.get('lon')

    if lat is None or lon is None:
        logging.error("❌ FATAL ERROR: 'lat' and 'lon' not found in the 'Parameters_or_Selectors' column.")
        sys.exit()

    logging.info(f"  > Latitude: {lat}")
    logging.info(f"  > Longitude: {lon}")

except (json.JSONDecodeError, TypeError):
    logging.error("FATAL ERROR: Could not parse Parameters_or_Selectors as JSON.")
    sys.exit()

# --- Step 3: Perform the two-step NWS API request ---
all_features = []
headers = {
    'User-Agent': os.environ.get('NWS_USER_AGENT', 'DAEN-NWS-Notebook/1.0 (contact)'),
    'Accept': 'application/geo+json'
}

logging.info("\n--- Starting NWS API Fetch ---")
try:
    point_lookup_url = f"{endpoint_url.strip('/')}/{lat},{lon}"
    logging.info(f"  > [Step 1] Requesting point lookup: {point_lookup_url}")
    response_step1 = requests.get(point_lookup_url, headers=headers, timeout=30)
    response_step1.raise_for_status()
    data_step1 = response_step1.json()
    forecast_url = data_step1.get('properties', {}).get('forecast')

    if not forecast_url:
        logging.error("❌ FATAL ERROR: Could not find a 'forecast' URL in the point lookup response.")
        sys.exit()

    logging.info(f"  > [Step 2] Fetching forecast data from: {forecast_url}")
    response_step2 = requests.get(forecast_url, headers=headers, timeout=30)
    response_step2.raise_for_status()
    data_step2 = response_step2.json()
    all_features = data_step2.get('properties', {}).get('periods', [])
    logging.info(f"✅ Successfully fetched {len(all_features)} forecast periods.")

except requests.exceptions.RequestException as e:
    logging.error(f"❌ FATAL ERROR: Could not fetch data from NWS API. Reason: {e}")
    sys.exit()

# --- Step 4: Process data into a DataFrame ---
logging.info("\n--- Processing Data into DataFrame ---")
if all_features:
    records = []
    for period in all_features:
        start_time_dt = pd.to_datetime(period.get('startTime'))
        records.append({
            'latitude': lat,
            'longitude': lon,
            'date': start_time_dt,
            'period': period.get('name'),
            'startTime': start_time_dt,
            'temp': f"{period.get('temperature')} {period.get('temperatureUnit')}",
            'wind': f"{period.get('windSpeed')} {period.get('windDirection')}",
            'forecast_short': period.get('shortForecast'),
            'forecast_detailed': period.get('detailedForecast')
        })
    df = pd.DataFrame(records)
    logging.info(f"✅ DataFrame created with {len(df)} records.")
else:
    logging.warning("❌ No forecast periods found. Cannot update Feature Layer. Exiting.")
    sys.exit()

if not USE_ARCGIS:
    df.to_csv(OUTPUT_CSV, index=False)

    def to_jsonable(val):
        if isinstance(val, pd.Timestamp):
            return val.isoformat()
        try:
            if pd.isna(val):
                return None
        except Exception:
            pass
        if hasattr(val, "item"):
            try:
                return val.item()
            except Exception:
                pass
        return val

    features = []
    if {"longitude", "latitude"}.issubset(df.columns):
        for _, row in df.iterrows():
            lon = row.get("longitude")
            lat = row.get("latitude")
            if pd.notna(lon) and pd.notna(lat):
                props = row.drop(labels=["longitude", "latitude"]).to_dict()
                props = {k: to_jsonable(v) for k, v in props.items()}
                features.append({
                    "type": "Feature",
                    "geometry": {"type": "Point", "coordinates": [float(lon), float(lat)]},
                    "properties": props
                })
    geojson = {"type": "FeatureCollection", "features": features}
    with open(OUTPUT_GEOJSON, "w", encoding="utf-8") as f:
        json.dump(geojson, f, ensure_ascii=False, indent=2)
    logging.info("Local outputs written: %s and %s", OUTPUT_CSV, OUTPUT_GEOJSON)


if USE_ARCGIS:
    # --- Step 5: Update the ArcGIS Online Feature Layer ---
    logging.info("\n--- Updating ArcGIS Online Feature Layer ---")
    try:
        logging.info("  > Getting Feature Layer Item")
        feature_layer_item = gis.content.get(ITEM_ID)
    
        if not feature_layer_item:
            logging.error("FATAL ERROR: Could not find the feature layer item. Check ITEM_ID / env vars.")
            sys.exit()

        target_layer = feature_layer_item.layers[LAYER_COUNT]
        logging.info(f"  > Accessing layer: '{target_layer.properties.name}'")

        logging.info("  > Clearing all existing records from the layer...")
        target_layer.delete_features(where="1=1")
        logging.info("  > Layer successfully cleared.")

        # --- Step 6: Prepare features for upload ---
        logging.info("\n--- Preparing Features for Upload ---")
    
        # FIX: Convert datetime objects to a standard string format before upload.
        # This prevents the 'Int64 to DateTime' conversion error.
        logging.info("  > Converting datetime columns to string format for AGOL compatibility...")
        df['date'] = df['date'].dt.strftime('%Y-%m-%d %H:%M:%S')
        df['startTime'] = df['startTime'].dt.strftime('%Y-%m-%d %H:%M:%S')
    
        features_to_add = []
        for index, row in df.iterrows():
            attributes = row.to_dict()
        
            # Clean up potential null values before creating the feature
            clean_attrs = {k: v for k, v in attributes.items() if pd.notna(v)}
        
            geometry = Point({
                "x": row['longitude'],
                "y": row['latitude'],
                "spatialReference": {"wkid": 4326}
            })
        
            new_feature = Feature(geometry=geometry, attributes=clean_attrs)
            features_to_add.append(new_feature)
    
        logging.info(f"✅ Prepared {len(features_to_add)} features for upload.")

        if features_to_add:
            logging.info(f"  > Appending {len(features_to_add)} new records to the layer...")
            add_result = target_layer.edit_features(adds=features_to_add)
        
            if add_result['addResults'] and all(res['success'] for res in add_result['addResults']):
                logging.info(f"✅✅✅ Update Complete! Successfully added {len(add_result['addResults'])} records.")
            else:
                logging.error("❌ FAILED to update Feature Layer. See details below:")
                logging.error(add_result)
        else:
            logging.info("  > No features to add.")
        
    except Exception as e:
        logging.error(f"❌ FATAL ERROR: An unexpected error occurred during the AGOL update. Reason: {e}")

    logging.info("\n--- Script Finished ---")